In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# Import ddm functions
from ddm.data_handling import read_file, export_data
from ddm.processing import ddm
from ddm.fitting import compute_AB, compute_ISF
from ddm.plotting import plotSingleExpFit, plotISF

### Steps

1. Load dataset. Supported formats are .tif, .nd2, and .lif.
1. From the resultant g(q, tau), determine A and B using `compute_AB` - at this point move away from dask arrays.
1. Run the ddm function on the loaded dataset
1. Determine the ISF for various q values
1. Run fitting procedures on these with a few models and show the output
1. TODO: Compare to traditional particle tracking algorithms

### Loading in Data


In [ ]:
data_file = "../data/test_dataset_drift.tif"
output_path = "../results/"

At this point additional user input may be required if the programme detects a multi-experiment file

In [ ]:
ddmData = read_file(data_file, delayed=True, chunk_size=5)

### Determining A and B

In [ ]:
# Calculation on the CPU using dask (GPU not yet supported)
A, B = compute_AB(ddmData)

In [ ]:
plt.plot(A[10:])
plt.axhline(B, color='r', linestyle='--')
plt.show()

In [ ]:
#TODO Save A,B

### Running the DDM Function
This part will determine the full DDM matrix as the square of the difference between Fourier transformed frames.

_This calculation can be performed on a CPU or a GPU. If a GPU is detected and CUDA and cupy are installed, the GPU will be selected._

In [ ]:
taus = np.arange(1,50)
ddmMatrix = ddm(ddmData, taus)

In [ ]:
for result in ddmMatrix:
    plt.plot(result, 'k', alpha=0.1)
plt.xlim(0, 100)
plt.show()

### Export ddmMatrix
If an output file already exists, an attempt is made to combine the datasets. This allows for the iterative calculation of (monotonically) increasing lag times.

In [ ]:
# Export ddm matrix
ddmMatrix = export_data(output_path, ddmMatrix, taus, data_file)

### Calculating the ISF

In [ ]:
isf = compute_ISF(ddmMatrix.data, A, B)

To access a specific q value (in units of 1/pixels) one can use regular indexing of the 0th axis of the ISF

In [ ]:
#zero q
isf[0]

#medium q
isf[len(isf)//2]

#last q
isf[len(isf)-1]

### Fitting/Plotting
An example to fit and plot to a single exponential. Supported fitting functions are double exponential and the Schultz model

In [ ]:
taus = ddmMatrix["tau"]
plotISF(isf, taus, q = len(isf)//2)
plotSingleExpFit(isf, taus)
plt.ylim(None, 1)
plt.show()